In [256]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import plotly.express as px
import seaborn as sns
from unidecode import unidecode

corr_xlsx = pd.read_excel('dados/corr/correspondencias.xlsx', converters={'cod_old':str,'cod_new':str})
corr = dict(zip(corr_xlsx.cod_old,corr_xlsx.cod_new))
cod_to_dsg = dict(zip(corr_xlsx.cod_new, corr_xlsx.dsg_new))
corr_concelhos_xlsx = pd.read_excel('dados/corr/correspondencias_old.xlsx', sheet_name = "concelhos", converters={'DICO':str,'Concelho':str})
corr_concelhos = dict(zip(corr_concelhos_xlsx.DICO, corr_concelhos_xlsx.Concelho))


def read_mai(ano):
    if ano >= 2009:
        df = pd.read_excel("dados/MAI/BDRE_Contagem_Eleitores_" + str(ano) + ".xls", sheet_name="Freguesia_Consulado", converters={"Codigo":str, 'Código':str, "Nac":int, "UE": int, "ER":int,"TOTAL":int})
        rename_dict = {"Codigo":"FREGUESIA", "Código":"FREGUESIA", "Nac":"NAC", "Distrito/Ilha/Continente > Concelho/País > Freguesia/Consulado":"Nome"}

        for old_col, new_col in rename_dict.items():
            if old_col in df.columns:
                df = df.rename(columns={old_col: new_col})
                
        df = df[["FREGUESIA", "Nome", "NAC"]]
        df = df[~df["FREGUESIA"].isna()]
        df = df.loc[df["FREGUESIA"].map(lambda x: int(str(x)[:2])) < 50]
        if ano == 2013:
            df["FREGUESIA"] = df["FREGUESIA"].apply(lambda x: x.replace('  ',''))
        df["FREGUESIA"] = df["FREGUESIA"].map(corr)
        df = df.groupby('FREGUESIA').agg({'Nome': 'first', 'NAC': 'sum'})
        df = df.rename(columns={"NAC": "PT_MAI_" + str(ano)[2:]})
    else:
        df = pd.read_excel("dados/MAI/BDRE_Contagem_Eleitores_" + str(ano) + ".xls", converters={"Codigo":str, 'Código': str, "Nac":int, "UE": int, "ER":int,"TOTAL":int}, skiprows=1)
        if ano == 2002:
            df = df.drop(columns=["Unnamed: 0", "AVEIRO"])
        else:  
            df = df.drop(columns=["CONTINENTE E REGIÕES AUTÓNOMAS", "Unnamed: 1"])
        df = df.rename(columns = {"Unnamed: 2" : "Concelho", "Unnamed: 3":"Nome"})

        column_names = df.columns.tolist()
        column_names[-1] = 'NAC'
        df.columns = column_names

        df['Concelho'] = df['Concelho'].fillna(method='ffill')
        df = df[df['Nome'].notna()]

        # create a list of values to keep
        concelhos = corr_xlsx["Concelho"].unique().tolist()
        mask = df['Concelho'].isin(concelhos)
        df = df.loc[mask].reset_index(drop=True)

        df['Nome'] = df['Nome'].str.upper()
        df["FREGUESIA"] = np.nan
        
        df["Nome"] = df["Nome"].apply(lambda x: unidecode(x))
        df["Concelho"] = df["Concelho"].apply(lambda x: unidecode(x))
        #compare_xlsx = corr_xlsx
        #compare_xlsx["Nome"] = df["Nome"].apply(lambda x: unidecode(x))
        #compare_xlsx["Concelho"] = df["Concelho"].apply(lambda x: unidecode(x))

        for idx, row in df.iterrows():
            select_df = corr_xlsx.loc[(corr_xlsx['Concelho'] == row["Concelho"]) & (corr_xlsx['dsg_old'] == row["Nome"])]
            if not select_df.empty:
                df.loc[idx, "FREGUESIA"] = select_df.iloc[0]["cod_new"]
                df.loc[idx, "Nome"] = select_df.iloc[0]["dsg_new"]

        df = df[df["Nome"] != "HAVANA"]
        df = df.groupby('FREGUESIA').agg({'Nome': 'first', 'NAC': 'sum'})
        df = df.rename(columns={"NAC": "PT_MAI_" + str(ano)[2:]})
        
    return df
        
portugal_districts = {
    '01': 'Aveiro',
    '02': 'Beja',
    '03': 'Braga',
    '04': 'Bragança',
    '05': 'Castelo Branco',
    '06': 'Coimbra',
    '07': 'Évora',
    '08': 'Faro',
    '09': 'Guarda',
    '10': 'Leiria',
    '11': 'Lisboa',
    '12': 'Portalegre',
    '13': 'Porto',
    '14': 'Santarém',
    '15': 'Setúbal',
    '16': 'Viana do Castelo',
    '17': 'Vila Real',
    '18': 'Viseu',
    '31': 'Madeira',
    '32': 'Madeira',
    '41': 'Açores',
    '42': 'Açores',
    '43': 'Açores',
    '44': 'Açores',
    '45': 'Açores',
    '46': 'Açores',
    '47': 'Açores',
    '48': 'Açores',
    '49': 'Açores'
}

pd.set_option('display.max_rows', 20)

nac21 = pd.read_excel('dados/dados_INE.xlsx', sheet_name="Nacionalidade2021", converters={'MUNICIPIO':str,'FREGUESIA':str}, skiprows=[0])
nac11 = pd.read_excel('dados/dados_INE.xlsx', sheet_name="Nacionalidade2011", converters={'MUNICIPIO_CAOP2013':str,'FREGUESIA_CAOP2013':str}, skiprows=[0])
nac01 = pd.read_excel('dados/dados_INE.xlsx', sheet_name="Nacionalidade2001", converters={'CONCELHO_EU02':str,'FREGUESIA_EU02':str}, skiprows=[0])
tipologias2014 = pd.read_excel('dados/tipologias2014.xlsx').rename(columns={"Código": "FREGUESIA"}).set_index("FREGUESIA")


nac21 = nac21[(nac21["FREGUESIA"] != '000000') & (nac21["GRUPO ETÁRIO"] == "2 18 ou mais anos")]
nac21 = nac21.rename(columns={"Designação CAOP2013": "FF_DSG", "Portuguesa":"PT", "Estrangeira":"EST", "Apátrida":"APA", "Total":"TOTAL"})
nac21 = nac21[["CC_DSG", "FREGUESIA", "FF_DSG", "PT"]]
nac21["FREGUESIA"] = nac21["FREGUESIA"].map(corr)
nac21['FF_DSG'] = nac21['FREGUESIA'].map(cod_to_dsg)
nac21 = nac21.groupby('FREGUESIA').agg({'CC_DSG': 'first', 'FF_DSG': 'first', 'PT': 'sum'})

nac11 = nac11[(nac11["FREGUESIA_CAOP2013"] != '000000') & (nac11["GRUPO ETÁRIO"] == "2 18 ou mais anos")]
nac11 = nac11.rename(columns={"Designação CAOP2013": "FF_DSG", "FREGUESIA_CAOP2013": "FREGUESIA", "Portuguesa":"PT", "Estrangeira":"EST", "Apátrida":"APA", "Total":"TOTAL"})
nac11 = nac11[["CC_DSG", "FREGUESIA", "FF_DSG", "PT"]]
nac11["FREGUESIA"] = nac11["FREGUESIA"].map(corr)
nac11['FF_DSG'] = nac11['FREGUESIA'].map(cod_to_dsg)
nac11 = nac11.groupby('FREGUESIA').agg({'CC_DSG': 'first', 'FF_DSG': 'first', 'PT': 'sum'})

nac01 = nac01[(nac01["FREGUESIA_EU02"] != '000000') & (nac01["GRUPO ETÁRIO"] == "2 18 ou mais anos")]
nac01 = nac01.rename(columns={"FREGUESIA_EU02": "FREGUESIA", "Portuguesa":"PT", "Estrangeira":"EST", "Apátrida":"APA", "Total":"TOTAL"})
nac01["FREGUESIA"] = nac01["FREGUESIA"].map(corr)
nac01['FF_DSG'] = nac01['FREGUESIA'].map(cod_to_dsg)
nac01 = nac01.groupby('FREGUESIA').agg({'CC_DSG': 'first', 'FF_DSG': 'first', 'PT': 'sum'})

nac11 = nac11.drop(['FF_DSG', 'CC_DSG'], axis=1)
nac01 = nac01.drop(['FF_DSG', 'CC_DSG'], axis=1)
nac21 = nac21.rename(columns={"TOTAL":"TOTAL_INE_21", "PT":"PT_INE_21"})
nac11 = nac11.add_suffix('_INE_11')
nac01 = nac01.add_suffix('_INE_01')



In [ ]:
tipologias2014 = tipologias2014.drop("Designação", axis=1)
tipologias2014["Distrito"] = tipologias2014.index.map(lambda x: portugal_districts.get(x[:2], 'Unknown'))

populacao = tipologias2014.merge(nac21, left_index=True, right_index=True).merge(nac11, left_index=True, right_index=True).merge(nac01, left_index=True, right_index=True)

#MAI
ele21 = read_mai(2021)
ele11 = read_mai(2011)

mai = ele21.merge(ele11,left_index=True, right_index=True)

#Comparacao
comp = populacao.merge(mai, left_index=True, right_index=True)

comp["diff_abs_21"] = comp["PT_MAI_21"] - comp["PT_INE_21"]
comp["diff_abs_21"] = comp["diff_abs_21"].astype(float)
comp["diff_abs_11"] = comp["PT_MAI_11"] - comp["PT_INE_11"]
comp["diff_rel_21"] = comp["diff_abs_21"] / comp["PT_INE_21"] * 100
comp["diff_rel_11"] = comp["diff_abs_11"] / comp["PT_INE_11"] * 100
comp['diff_rel_21'] = comp['diff_rel_21'].astype(float).round(1)
comp['diff_rel_11'] = comp['diff_rel_11'].astype(float).round(1)
comp_APU = comp[comp["TIPAU"] == "APU"]
comp_AMU = comp[comp["TIPAU"] == "AMU"]
comp_APR = comp[comp["TIPAU"] == "APR"]

def print_mapa(var="diff_rel_21", ano = 2021):
    district_stats = comp.groupby('Distrito')[var].apply(lambda x: pd.Series({'weighted_mean_diff': (x * comp.loc[x.index, 'PT_INE_21']).sum() / comp.loc[x.index, 'PT_INE_21'].sum()}))
    district_stats = pd.DataFrame(district_stats).reset_index().drop(columns=["level_1"]).set_index("Distrito")

    mapa = gpd.read_file('PRT_adm1.shp')
    mapa = mapa.replace({"Azores":"Açores"})
    mapa = mapa.merge(district_stats, left_on='NAME_1', right_index=True)

    # Create the map
    fig, ax = plt.subplots(figsize=(12, 8))
    mapa.plot(column=var, cmap='OrRd', legend=True, ax=ax)
    ax.set_title('Erro relativo médio por distrito (poderado) ' + str(ano), fontsize=16)
    ax.set_axis_off()

    # Add labels to the map with the average population for each district
    for idx, row in mapa.iterrows():
        plt.annotate(text=f"{row[var]:.0f}", xy=row['geometry'].centroid.coords[0], horizontalalignment='center', fontsize=15)
    plt.show()
    
def print_boxplot(var = "diff_rel_21", titulo = "Erro relativo p/TIPAU - 2021"):
    data = [comp[var], comp_APU[var], comp_AMU[var], comp_APR[var]]

    # Plot the boxplots
    fig, ax = plt.subplots()
    boxplots = ax.boxplot(data)

    # Add x-axis labels
    ax.set_title(titulo)
    ax.set_xticklabels(['Todas', "APU", "AMU", "APR"])
    ax.set_ylabel("%")
    ax.set_ylim([-80, 180])
    # Show the plot
    plt.show()

In [ ]:
print_boxplot()

In [ ]:
#cmap = {'APR': 'green', 'APU': 'red', 'AMU': 'blue'}

fig = px.scatter(comp_APU, x='PT_INE_21', y='diff_rel_21', hover_data=['FF_DSG', "PT_MAI_21"])

# Show the plot
fig.show()

In [ ]:
comp_APU.head()

In [ ]:
# ADICIONAR VARIÁVEIS (MÉDIA PONDERADA PARA AS UNIÕES DE FREGUESIAS (EX: EDUCAÇÃO))
df = pd.DataFrame({
    'Index': ['A', 'A', 'B', 'B', 'C'],
    'A': [1, 2, 3, 4, 5],
    'B': [10, 20, 30, 40, 50]
})

weighted_mean = lambda x: np.average(x, weights=df.loc[x.index, "A"])

grouped_df = df.groupby('Index').agg(A=("A", "sum"),B_weighted=("B", weighted_mean))


In [ ]:
#GRÁFICO COMPARAÇÃO POPULAÇÃO RECENSEADA 

years_mai = list(range(2008, 2022))
mai_list = []
for year in range(2008, 2022):
    df = read_mai(year)
    mai = df["PT_MAI_" + str(year)[2:]].sum()
    print(str(year) + ": " + str(mai))
    mai_list.append(mai)


years_ine = [2011, 2021]
ine_list = [8166640,  8079525]

# create two series objects with different numbers of observations
mai = pd.Series(mai_list, index=years_mai)
ine = pd.Series(ine_list, index=years_ine)

# reindex both series to have a common index
common_index = pd.RangeIndex(start=2010, stop=2023)
mai = mai.reindex(common_index)
ine = ine.reindex(common_index)


df = pd.concat([mai, ine], axis=1)
df.columns = ['Series 1', 'Series 2']

# plot both series using Seaborn
sns.lineplot(data=df)
plt.show()


In [259]:
for year in range(2001, 2022):
    df = read_mai(year)
    mai = df["PT_MAI_" + str(year)[2:]].sum()
    print(str(year) + ": " + str(mai))
    print(df.shape[0])
    

2001: 8695958
3075
2002: 8670378
3075
2003: 8687945
3075
2004: 8750036
3075
2005: 8791097
3075
2006: 8792187
3075
2007: 8770298
3075
2008: 9462645
3075
2009: 9371653
3075
2010: 9427654
3075
2011: 9454640
3075
2012: 9465089
3075
2013: 9471211
3075
2014: 9448259
3075
2015: 9439510
3075
2016: 9387932
3075
2017: 9360940
3075
2018: 9342202
3075
2019: 9318394
3075
2020: 9314947
3075
2021: 9298389.0
3075


In [257]:
mai06 = read_mai(2006)
mai21 = read_mai(2021)

In [258]:
print(len(set(mai21.index) - set(mai06.index)))

set(mai21.index) - set(mai06.index)


0


set()

In [120]:
mai01.head()

,Nome,PT_MAI_01
FREGUESIA,,
010103,AGUADA DE CIMA,3289
010109,FERMENTELOS,2383
010112,MACINHATA DO VOUGA,2913
010119,VALONGO DO VOUGA,4199
010121,UNIÃO DAS FREGUESIAS DE ÁGUEDA E BORRALHA,11775


In [ ]:
#430210 VERIFICAR SE ESTÁ NOS ANOS TODOS

In [251]:
        ano = 2006
        df = pd.read_excel("dados/MAI/BDRE_Contagem_Eleitores_" + str(ano) + ".xls", converters={"Codigo":str, 'Código': str, "Nac":int, "UE": int, "ER":int,"TOTAL":int}, skiprows=1)
        if ano == 2002:
            df = df.drop(columns=["Unnamed: 0", "AVEIRO"])
        else:  
            df = df.drop(columns=["CONTINENTE E REGIÕES AUTÓNOMAS", "Unnamed: 1"])
        df = df.rename(columns = {"Unnamed: 2" : "Concelho", "Unnamed: 3":"Nome"})

        column_names = df.columns.tolist()
        column_names[-1] = 'NAC'
        df.columns = column_names

        df['Concelho'] = df['Concelho'].fillna(method='ffill')
        df = df[df['Nome'].notna()]

        # create a list of values to keep
        concelhos = corr_xlsx["Concelho"].unique().tolist()
        mask = df['Concelho'].isin(concelhos)
        df = df.loc[mask].reset_index(drop=True)

        df['Nome'] = df['Nome'].str.upper()
        df["FREGUESIA"] = np.nan
        
        df["Nome"] = df["Nome"].apply(lambda x: unidecode(x))
        df["Concelho"] = df["Concelho"].apply(lambda x: unidecode(x))
        #compare_xlsx = corr_xlsx
        #compare_xlsx["Nome"] = df["Nome"].apply(lambda x: unidecode(x))
        #compare_xlsx["Concelho"] = df["Concelho"].apply(lambda x: unidecode(x))

        for idx, row in df.iterrows():
            select_df = corr_xlsx.loc[(corr_xlsx['Concelho'] == row["Concelho"]) & (corr_xlsx['dsg_old'] == row["Nome"])]
            if not select_df.empty:
                df.loc[idx, "FREGUESIA"] = select_df.iloc[0]["cod_new"]
                df.loc[idx, "Nome"] = select_df.iloc[0]["dsg_new"]

        df = df[df["Nome"] != "HAVANA"]
        #df = df.groupby('FREGUESIA').agg({'Nome': 'first', 'NAC': 'sum'})
        #df = df.rename(columns={"NAC": "PT_MAI_" + str(ano)[2:]})

In [252]:
df[df["FREGUESIA"].isna()]

,Concelho,Nome,NAC,FREGUESIA


In [197]:
df = pd.read_excel("dados/MAI/BDRE_Contagem_Eleitores_" + str(2001) + ".xls", converters={"Codigo":str, 'Código': str, "Nac":int, "UE": int, "ER":int,"TOTAL":int}, skiprows=1)
df = df.drop(columns=["CONTINENTE E REGIÕES AUTÓNOMAS", "Unnamed: 1"])
df = df.rename(columns = {"Unnamed: 2" : "Concelho", "Unnamed: 3":"Nome"})
df['Concelho'] = df['Concelho'].fillna(method='ffill')
column_names = df.columns.tolist()
column_names[-1] = 'NAC'
df.columns = column_names
df['Concelho'] = df['Concelho'].fillna(method='ffill')
df = df[df['Nome'].notna()]

#concelhos = corr_xlsx["Concelho"].unique().tolist()
#mask = df['Concelho'].isin(concelhos)
#df = df.loc[mask].reset_index(drop=True)